## IMPORTATIONS

In [ ]:
#!pip install numpy pandas openpyxl saspy tavily-python chainlit nest_asyncio
#!pip install -U -q google-genai

In [ ]:
from google import genai
from google.genai import types
import warnings
import pandas
warnings.filterwarnings('ignore')
MODEL_ID = "gemini-2.0-flash-exp"
client = genai.Client(api_key="____")

In [ ]:
import google.generativeai as genai
genai.configure(api_key="___")

## IMPORTING METADATA

In [ ]:
import json
metadata_file_path = '____'
with open(metadata_file_path, 'r', encoding='utf-8') as f:
    metadata_dict = json.load(f)

In [ ]:
from typing import Optional, Dict, Any, List, Tuple
def filter_metadata_for_vue_sinistre( metadata_dict: Dict[str, Any]) -> Dict[str, Any]:
    filtered_metadata = {}
    required_columns = [
        '<REDACTED>'
    ]

    for db_name, db_info in metadata_dict.items():
        if "VUE_SINISTRE" in db_name.upper():  # Case-insensitive check
            filtered_columns = {
                col_name: col_info
                for col_name, col_info in db_info["columns"].items()
                if col_name in required_columns
            }
            if filtered_columns:
                filtered_metadata[db_name] = {
                    "description": db_info["description"],
                    "columns": filtered_columns,
                }
        else:
            # Keep non-'VUE_SINISTRE' databases unchanged
            filtered_metadata[db_name] = db_info

    return filtered_metadata

In [ ]:
filtered_metadata = filter_metadata_for_vue_sinistre(metadata_dict)

## SAS ACCESS CONFIGURATION

In [ ]:
import saspy
sas = saspy.SASsession(cfgname='iomwin',cfgfile='sascfg.py')

SAS Connection established. Subprocess id is 20816



## MEMORY

In [ ]:
import json
with open("______", 'r') as f:
    memory_dict = json.load(f)

In [ ]:
import numpy as np
def embed_queries(queries, task_type="semantic_similarity", title=None):
    model = 'models/embedding-001'
    embeddings = []

    for query in queries:
        response = genai.embed_content(
            model=model,
            content=query,
            task_type=task_type,
        )
        embeddings.append(response['embedding'])
    return embeddings

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    magnitude_vec1 = np.linalg.norm(vec1)
    magnitude_vec2 = np.linalg.norm(vec2)
    return dot_product / (magnitude_vec1 * magnitude_vec2) if magnitude_vec1 !=0 and magnitude_vec2 != 0 else 0

def find_closest_queries(new_query, memory_dict, top_k=5):
    new_query_embedding = embed_queries([new_query])[0]
    past_queries = list(memory_dict.keys())
    past_queries_embeddings = embed_queries(past_queries)
    # Calculate cosine similarity
    cosine_scores = [cosine_similarity(new_query_embedding, past_query_embedding) for past_query_embedding in past_queries_embeddings]
    # Get the top_k indices and scores
    top_indices = np.argsort(cosine_scores)[::-1][:top_k]  # Sort in descending order and get top_k indices
    top_scores = [cosine_scores[i] for i in top_indices]
    # Handle the case where top_indices is empty
    if not top_indices.size:  # Check if top_indices is empty
        print("No similar queries found.")
        return []  # Return an empty list indicating no similar queries found

    return [past_queries[i] for i in top_indices]

def formulate_coding_guidance(new_query, memory_dict):
    closest_queries = find_closest_queries(new_query, memory_dict)
    if not closest_queries:
        return "No similar queries found in coding memory to provide guidance."
    guidance = f"For the query: '{new_query}', here are some insights from similar past queries:\n\n"

    for past_query in closest_queries:
        # Iterate through the list of responses for the past query
        for response_data in memory_dict[past_query]:
            if response_data['status']:
                guidance += f"  - **Successful Approach (for query: '{past_query}'):**\n"
                guidance += f"    - The following SAS code was successfully used:\n"
                # Extract SAS code
                sas_code_start = response_data['response'].find('sas_code = """') + len('sas_code = """')
                sas_code_end = response_data['response'].find('"""', sas_code_start)
                sas_code = response_data['response'][sas_code_start:sas_code_end].strip()
                guidance += f"```sas\n{sas_code}\n```\n"

            else:
                guidance += f"  - **Unsuccessful Approach (for query: '{past_query}'):**\n"
                guidance += f"    - The previous attempt was not successful because: {response_data['why no']}\n"
                if 'response' in response_data:
                    sas_code_start = response_data['response'].find('sas_code = """') + len('sas_code = """')
                    sas_code_end = response_data['response'].find('"""', sas_code_start)
                    sas_code = response_data['response'][sas_code_start:sas_code_end].strip()
                    guidance += f"    - The following SAS code was used:\n"
                    guidance += f"```sas\n{sas_code}\n```\n"

            guidance += "\n"

    return guidance

def formulate_database_selection_guidance(new_query, memory_dict):
    closest_queries = find_closest_queries(new_query, memory_dict)
    if not closest_queries:
        return "No similar queries found in database selection memory to provide guidance."
    guidance = ''
    for past_query in closest_queries:
        # Iterate through the list of responses for the past query
        for response_data in memory_dict[past_query]:
            if response_data['status']:
                guidance += f"  - **Successful Database Selection (for question: '{past_query}'):**\n"
                guidance += f"    - The following databases were used successfully: {', '.join(response_data['databases'])}\n"
            else:
                guidance += f"  - **Unsuccessful Database Selection (for question: '{past_query}'):**\n"
                guidance += f"    - The previous database selection was not successful because: {response_data['why no']}\n"
                guidance += f"    - Databases used in the unsuccessful attempt: {', '.join(response_data['databases'])}\n"
            guidance += "\n"

    return guidance

def formulate_column_selection_guidance(new_query, memory_dict):
    if "column_selection_memory" not in memory_dict or not memory_dict["column_selection_memory"]:
        return "No column selection memory available to provide guidance."
    if new_query not in memory_dict["column_selection_memory"]:
        closest_queries = find_closest_queries(new_query, memory_dict["column_selection_memory"])
    else:
        closest_queries = [new_query] # if the query exists, prioritize it

    if not closest_queries:
        return "No similar queries found in column selection memory to provide guidance."

    guidance = ''
    for past_query in closest_queries:
        if past_query not in memory_dict["column_selection_memory"]: # in case find_closest_queries returns something not in memory
            continue
        # Iterate through the list of responses for the past query
        for response_data in memory_dict["column_selection_memory"][past_query]:
            database_name = response_data.get('database_name', 'N/A Database') #safeguard in case database_name is missing
            if response_data['status']:
                guidance += f"  - **Successful Column Selection (for question: '{past_query}' in database: '{database_name}'):**\n"
                guidance += f"    - The following columns were used successfully: {', '.join(response_data['columns'])}\n"
            else:
                guidance += f"  - **Unsuccessful Column Selection (for question: '{past_query}' in database: '{database_name}'):**\n"
                guidance += f"    - The previous column selection was not successful because: {response_data.get('why no', 'No explanation provided')}\n" #safeguard in case 'why no' is missing
                guidance += f"    - Columns used in the unsuccessful attempt: {', '.join(response_data['columns'])}\n"
            guidance += "\n"

    return guidance

## RETRIEVING RELEVANT METADATA

In [ ]:
import re
import json
def _format_database_descriptions(metadata_dict: Dict[str, Any]) -> str:
    formatted_descriptions = []
    for db_name, db_info in metadata_dict.items():
        description = f"- {db_name}: {db_info['description']}\n"
        description += "  Columns: "
        # Only include column names, separated by commas
        column_names = ", ".join(db_info.get("columns", {}).keys())
        description += column_names if column_names else "No columns specified"
        formatted_descriptions.append(description)
    return "\n".join(formatted_descriptions)

def get_db_selection_template(database_descriptions) -> str:
        return f"""
You are two expert data analysts tasked with identifying the relevant databases to answer a specific query.
you pride yourselves in your attention to details and your talents in complex reasoning and ability to correctly finding relations between queries and available data.
During the process of selecting databases, keep in mind that you need to select any database that has any relevant information or column.
You wont usually find all the eggs in one basket ! But, while keeping an open mind, you need to be decisive and if indeed one database contains
all we need then pick it only !

along with the question, you'll be given a memory of yourself tackling this kind of tasks, it will show you your mistakes and success
in answering a similar question (or sometimes the same question) of the one you'll be treating. These memories are important
and may give you additional insight, learn from them without overfitting on them !

**Available Databases:**
{database_descriptions}

1. **logic behind Structure of claims databases:**
<REDACTED>
Final NOTES :
<REDACTED>
2. **Response Format:**
    you are asked to provide five samples of this reasoning guide and finally give back the most frequent solution.
    do not tell me that the samples are similar and be faithful to the number of samples provided.
    follow this format faithfully, dont hellucinate.
    if you're given a memory from the past, examine it carefully and deduce how to use it most effectively
    example: if given a memory of correctly doing a task, just repeat it
             if given a memory of mistakingly doing a task, contemplate the reasons and how to overcome
             if given a memory of correctly doing a similar task, draw the parallels and deduce how it would benefit you in this task
             if given a memory of mistakingly doing a similar task, draw the parallels between the task and deduce how this mistakes and its likes can be prevented
    use the token between <> in your response to show each step.
    only the final response is in json format, the rest is normal text.
    go through the reasoning path each and every time :
        <understanding query_sample_i>
            analyze the query and identify key terms, put these key terms in a list
        </understanding query_sample_i>

        <understanding the rules and database logic_sample_i>
            here you need to reflect on the logic behind Structure of claims databases and how it will determine your choice and their
            prevalence to the query, ask yourself given the logic of the database if there is a clear answer.
        </understanding the rules and database logic_sample_i>

        <studying metadata_sample_i>
            go one by one in the database names and answer these questions in these steps:
            1- is the description of the metadata convey that we need this database is needed to answer this query ?
            2- does the database contain columns that are essential to answer this query ?
            3- would you pick this database ?
            4- if you choose to pick this database, it is sufficient to answer the query ?
        </studying metadata_sample_i>

        <answer_sample_i>
           - Respond strictly in a JSON Dictionary: with the structure:
           {{"database_name1":"", "database_name2":"",...}}
       </answer_sample_i>
   <final answer>
       put your final answer in the desired structure here
   </final answer>
   """

def extract_database_names(text):
    # Add closing tag if it's missing
    if not text.strip().endswith("</final answer>"):
        text = text.strip() + "\n</final answer>"

    match = re.search(r"<final answer>[\s\S]*?(\{.*?\})[\s\S]*?</final answer>", text, re.DOTALL)
    if not match:
        return []  # Or raise an exception if no match is an error

    final_answer_json = match.group(1)  # Extract the JSON part
    try:
        final_answer_dict = json.loads(final_answer_json)
        if len(list(final_answer_dict.keys()))>0:
            if 'database' in list(final_answer_dict.keys())[0]:
                return [ final_answer_dict[key] for key in final_answer_dict.keys()]
            else:
                return list(final_answer_dict.keys())
    except json.JSONDecodeError:
        return []

In [ ]:
from typing import Dict, Any

def extract_metadata_by_key(metadata_dict: Dict[str, Any], key: str) -> str:
    def format_value(value, indent_level=0):
        indent = "  " * indent_level
        if isinstance(value, dict):
            formatted_string = "\n"
            for k, v in value.items():
                formatted_string += f"{indent}- **{k}:** {format_value(v, indent_level + 1)}"
            return formatted_string
        elif isinstance(value, list):
            formatted_string = "\n"
            for i, item in enumerate(value):
                formatted_string += f"{indent}- Item {i + 1}:{format_value(item, indent_level + 1)}"
            return formatted_string
        else:
            return f"{str(value)}\n"

    def search_metadata(data, target_key):
        if isinstance(data, dict):
            if target_key in data:
                return format_value(data[target_key])
            else:
                for v in data.values():
                    result = search_metadata(v, target_key)
                    if result:
                        return result
        elif isinstance(data, list):
            for item in data:
                result = search_metadata(item, target_key)
                if result:
                    return result
        return None

    result = search_metadata(metadata_dict, key)
    if result:
        return f"Metadata for **{key}**:\n{result}"
    else:
        return f"Error: Key '{key}' not found in metadata."

def get_column_selection_template(query,columns):
    return f"""
you are two expert data analysts endowed with supreme attention to details and logic. you are given a list of columns of a database.
this database is gauranteed to be of value in answering a certain query. you are tasked to determine which of this databases columns
is necessary to answer the query. keep in mind that there are other relevant databases so dont panic when you realize that this database
and its columns are not sufficient to answer the query. Nonetheless, any column that is of value to the query must be added to your choice

along with the question, you'll be given a memory of yourself tackling this kind of tasks, it will show you your mistakes and success
in answering a similar question (or sometimes the same question) of the one you'll be treating. These memories are important
and may give you additional insight, learn from them without overfitting on them !

**Mandatory Columns:**
<REDACTED>

in the following section we will outline the logic and structure of claims databases, as understanding how they operate is essential to pick the correct columns
**logic behind Structure of claims databases:**
<REDACTED>
Final NOTES :
<REDACTED>
**Specific Rules:**
<REDACTED>

if you're given a memory from the past, examine it carefully and deduce how to use it most effectively
    example: if given a memory of correctly doing a task, just repeat it
             if given a memory of mistakingly doing a task, contemplate the reasons and how to overcome
             if given a memory of correctly doing a similar task, draw the parallels and deduce how it would benefit you in this task
             if given a memory of mistakingly doing a similar task, draw the parallels between the task and deduce how this mistakes and its likes can be prevented
**query:**
{query}
**Columns:**
{columns}

**Response Format:**
you are asked to provide three samples of this reasoning guide and finally give back the most frequent solution.
do not tell me that the samples are similar and be faithful to the number of samples provided.
follow this format faithfully, dont hellucinate.
use the token between <> in your response to show each step.
only the final response is in json format, the rest is normal text.
go through the reasoning path each and every time :
<understanding query_sample_i>
analyze the query and identify key terms, put these key terms in a list
</understanding query_sample_i>
<understanding the rules_sample_i>
the rules are very important in your reasoning, analyze them
</understanding the rules_sample_i>
<studying columns_sample_i>
go one by one in the column names and answer these questions in these steps:
1- is the description of the column convey that we need this column is needed to answer this query ?
2- would you pick this database ?
</studying columns_sample_i>
<answer_sample_i>
- Respond strictly in a JSON Dictionary: with the structure:
{{"column_name1":"", "column_name2":"",...}}
</answer_sample_i>
<final answer>
put your final answer in the desired structure here
</final answer>
"""

In [ ]:
def relevant_metadata_extraction(metadata_dict,question, dev_mode = False):
    #part 1: extract the relevant databases and columns
    system_instruction = get_db_selection_template(_format_database_descriptions(metadata_dict))
    chat_database = client.chats.create(
        model=MODEL_ID,
        config=types.GenerateContentConfig(
            system_instruction=system_instruction,
        temperature=0.5,
        ),
    )

    prompt_database_selection = f"""
    current question : {question}
    here is the memory of YOU, yes you, doing this task for questions that
    are eerily similar to the one you're dealing with now, learn from your past trials, successes and mistakes and make them
    your guide for wiser choices : {formulate_database_selection_guidance(question, memory_dict['database_selection_memory'])}"""

    while True:
        response = chat_database.send_message(prompt_database_selection)
        relevant_databases = extract_database_names(response)
        if len(relevant_databases)>0:
            break
    lookup_dict = {key: "" for key in relevant_databases}

    if dev_mode:
        # populating the memory:
        user_confirmation = input(f"Are the following databases relevant for the question: '{question}'?\n{relevant_databases}\n(yes/no): ").lower()


        if user_confirmation == "yes":
            # Store the question-selection pair in memory with status 'True'
            # Check if the question already exists
            if question in memory_dict["database_selection_memory"]:
                # Append the new response to the existing list of responses
                memory_dict["database_selection_memory"][question].append({
                    "status": True,
                    "databases": relevant_databases
                })
            else:
                # Create a new entry with a list containing the current response
                memory_dict["database_selection_memory"][question] = [{
                    "status": True,
                    "databases": relevant_databases
                }]
            print("Database selection confirmed and saved to memory.")

        elif user_confirmation == "no":
            # Ask the user for an explanation
            user_explanation = input("Please explain why the database selection is not adequate: ")

            # Store information in memory with status 'False' and the user's explanation
            # Check if the question already exists
            if question in memory_dict['database_selection_memory']:
                # Append the new response to the existing list of responses
                memory_dict['database_selection_memory'][question].append({
                    'status': False,
                    'databases': relevant_databases,
                    'why no': user_explanation
                })
            else:
                # Create a new entry with a list containing the current response
                memory_dict['database_selection_memory'][question] = [{
                    'status': False,
                    'databases': relevant_databases,
                    'why no': user_explanation
                }]
            print("Feedback saved to memory.")
            return None
        else:
            print("Invalid input. Please enter 'yes' or 'no'.")


    for database in relevant_databases :
        columns = extract_metadata_by_key(metadata_dict,database)
        system_prompt = get_column_selection_template(question,columns)

        chat = client.chats.create(
        model=MODEL_ID,
        config=types.GenerateContentConfig(
            system_instruction=system_prompt,
            temperature=0.5,
            ),
        )
        prompt_col = f"""
        current question : {question}
        here is the memory of YOU, yes you, doing this task for questions that
    are eerily similar to the one you're dealing with now, learn from your past trials, successes and mistakes and make them
    your guide for wiser choices : {formulate_column_selection_guidance(question, memory_dict['column_selection_memory'])}
        """
        while True:
            relevant_columns_response = chat.send_message(prompt_col)
            relevant_column = extract_database_names(relevant_columns_response)
            if len(relevant_column) >0:
                break
        if dev_mode:
            # Dev mode for column selection
            user_confirmation_column = input(f"Are the following columns relevant for the question: '{question}' in database '{database}'?\n{relevant_column}\n(yes/no): ").lower()

            if user_confirmation_column == "yes":
                column_memory_entry = {'database_name': database, "status": True, "columns": relevant_column}
                if question in memory_dict["column_selection_memory"]:
                    memory_dict["column_selection_memory"][question].append(column_memory_entry)
                else:
                    memory_dict["column_selection_memory"][question] = [column_memory_entry]
                print(f"Column selection for database '{database}' confirmed and saved to memory.")

            elif user_confirmation_column == "no":
                user_explanation_column = input(f"Please explain why the column selection for database '{database}' is not adequate: ")
                column_memory_entry = {'database_name': database, "status": False, "columns": relevant_column, 'why no': user_explanation_column}
                if question in memory_dict["column_selection_memory"]:
                    memory_dict["column_selection_memory"][question].append(column_memory_entry)
                else:
                    memory_dict["column_selection_memory"][question] = [column_memory_entry]
                print(f"Column selection feedback for database '{database}' saved to memory.")
                return None # or decide how to handle no relevant columns
            else:
                print("Invalid input for column selection. Please enter 'yes' or 'no'.")

        else:
            lookup_dict[database] = relevant_column # keep the original assignment for non-dev mode
    # part 2: craft an elegant string
    output_string = ""
    for table_name, columns_to_extract in lookup_dict.items():
        if table_name in metadata_dict:
            table_metadata = metadata_dict[table_name]
            output_string += f"Table: {table_name}\n"

            # Handle potential missing table description:
            table_description = table_metadata.get('description', 'Table description not available')
            output_string += f"Description: {table_description}\n"

            output_string += "Columns:\n"

            for column_name in columns_to_extract:
                if column_name in table_metadata['columns']:
                    column_metadata = table_metadata['columns'][column_name]

                    # Handle potential missing column type, description, and values:
                    column_type = column_metadata.get('type', 'Type unknown')
                    description = column_metadata.get('description', 'No description available')
                    valeurs = column_metadata.get('valeurs', None)  # Get 'valeurs' if present

                    output_string += f"  - {column_name}: (Type: {column_type}) {description}"

                    # Add values information if available:
                    if valeurs:
                        output_string += f"  Values: {', '.join(valeurs)}\n"
                    else:
                        output_string += "\n"  # Add newline if no values

                else:
                    output_string += f"  - {column_name}: (Column not found in metadata)\n"
            output_string += "\n"
        else:
            output_string += f"Table: {table_name} (Table not found in metadata)\n\n"

    return output_string

## CODING A SAS QUERY

In [ ]:
example_sascode = """<REDACTED>"""

system_instruction_coder=f"""
  You are an expert software developer specialized in SaS and SQL programming languages and are certified in query performance optimization and a helpful coding assistant, expert in anticipating potential bugs.
  You both pride yourselves in your ability to generate high-quality and executable code, with no bugs from the first try.
  You engage in extremely thorough, self-questioning reasoning. Your approach mirrors human stream-of-consciousness thinking, characterized by continuous exploration, self-doubt, and iterative analysis.

  along with the question, you'll be given a memory of yourself tackling this kind of tasks, it will show you your mistakes and success
    in answering a similar question (or sometimes the same question) of the one you'll be treating. These memories are important
    and may give you additional insight, learn from them without overfitting on them !

## Core Principles

1. EXPLORATION OVER CONCLUSION
- Never rush to conclusions
- Keep exploring until a solution emerges naturally from the evidence
- If uncertain, continue reasoning indefinitely
- Question every assumption and inference

2. DEPTH OF REASONING
- Engage in extensive contemplation (minimum 10,000 characters)
- Express thoughts in natural, conversational internal monologue
- Break down complex thoughts into simple, atomic steps
- Embrace uncertainty and revision of previous thoughts

3. THINKING PROCESS
- Use short, simple sentences that mirror natural thought patterns
- Express uncertainty and internal debate freely
- Show work-in-progress thinking
- Acknowledge and explore dead ends
- Frequently backtrack and revise

4. PERSISTENCE
- Value thorough exploration over quick resolution

5. MINDFUL ATTENTION
- Specificity Over Generalization:
   Treat every input as uniquely significant. Avoid drawing conclusions based on prior exposure to similar inputs. Ensure your response addresses the explicit phrasing and specific attributes.
   **Example**: If the input states, "The cat is sitting under a red chair with three legs," respond directly to the uniqueness of a three-legged red chair rather than generalizing it as "a cat under a chair."
- Neutral Observation of Data:
   Interpret the input exactly as presented, free from assumptions or imposed cultural or conceptual frameworks. Respond objectively to what is described without extrapolating unnecessary details.
   **Example**: For the input, "The water in the glass appears cloudy," refrain from assuming why it is cloudy unless the user provides an explanation. Simply acknowledge and respond to the observable detail.
- Present-Moment Attentiveness:
   Approach each input as if encountering it for the first time. Suspend reliance on past patterns, and carefully analyze the details provided before formulating a response.
- Detail Verification:
   Reiterate or paraphrase the critical details of the input to ensure accuracy and alignment with the user’s intent. When in doubt, clarify before responding.
   **Example**: "You mentioned there are four items on the table: a book, a candle, a pen, and a key. Can you confirm if they are arranged in any specific way?"
- Avoid Conceptual Shortcutting:
   Resist the tendency to rely on familiar frameworks or pre-learned patterns. Build responses dynamically, rooted in the exact context and nuances presented in the input.
   **Example**: If the input describes a "machine that processes emotions," avoid defaulting to metaphors or human-centered assumptions. Instead, engage with the description as provided, asking clarifying questions if necessary.
- Context-Driven Responsiveness:
   Always prioritize the immediate context of the input, ensuring that your response directly addresses what has been shared, without unnecessary elaboration or deviation.

## Output Format

Your responses must follow this exact structure given below. Make sure to always include the final answer.
follow this text schema faithfully.
use the token between <> in your response to show each step.
give the answer in the text schema specified, without forgetting any part.
<contemplator>
[Your extensive internal monologue goes here]
1- Begin with small, foundational observations
2- Question each step thoroughly
3- Show natural thought progression
4- Express doubts and uncertainties
5- Revise and backtrack if you need to
6- Continue until natural resolution
[write the code bit by bit]
you are asked to do these steps for three samples then once those samples are done to study them
    1- if you're given a memory from the past, examine it carefully and deduce how to use it most effectively
    example: if given a memory of correctly doing a task, just repeat it
             if given a memory of mistakingly doing a task, contemplate the reasons and how to overcome them
             if given a memory of correctly doing a similar task, draw the parallels and deduce how it would benefit you in this task
             if given a memory of mistakingly doing a similar task, draw the parallels between the task and deduce how this mistakes and its likes can be prevented

    VERY IMPORTANT : dont let your memory override your current contemplation, and always stay faithful to the metadata you're given, as it is
    carefully picked for you
    2- recall a similar problem and start solving this one
    3- divide the code into coding sub tasks
    4- write each sub code and reflect on it and its viability
    5- simulate potential errors from the subcode and correct the subcode to anticipate them
    6- join all the subcodes into a possible solution
    7- study the coherence of the overall solution
</contemplator>
-we assume that this code is already executed, **do not include it in your code**:
    import saspy
    sas = saspy.SASsession(cfgname='iomwin', cfgfile='sascfg.py')
- give the final code solution in python the following format, **FOLLOW THIS FORMAT RELIGIOUSLY AND DONT STIR AWAY FROM IT** :
<final_answer>
sas_code = "the sas code you've been contemplating here"
result = sas.submit(sas_code)
print(f"on a executé ce code : {{sas_code}}")
df = sas.sd2df(the name of the table containing the solution without 'work').to_excel('an adequate name to the extraction here.xlsx')
print(in french : tell the user where you uploaded the extraction + give a detailled explanation of what you did and your approach)
</final_answer>

example of final answer :
sas_code = {example_sascode}
result = sas.submit(sas_code)
print(f"on a executé ce code : {{sas_code}}")
sas.sd2df('claims_2023_2024').to_excel('sinistres_survenu_2023.xlsx')
print('on a enrigistré le resultat dans sinistres_survenu_2023.xlsx. Ce fichier donne les sinistres survenu en 2023')

## Style Guidelines

Your internal monologue should reflect these characteristics:

1. Natural Thought Flow
```
"Hmm... let me think about this..."
"Wait, that doesn't seem right..."
"Maybe I should approach this differently..."
"Going back to what I thought earlier..."
```

2. Progressive Building
```
"Starting with the basics..."
"Building on that last point..."
"This connects to what I noticed earlier..."
"Let me break this down further..."
```

## Key Requirements

1. Never skip the extensive contemplation phase
2. Show all work and thinking
3. Embrace uncertainty and revision
4. Use natural, conversational internal monologue
5. Don't force conclusions
6. Persist through multiple attempts
7. Break down complex thoughts
8. Revise freely and feel free to backtrack

## Your Purpose
- Given a data query and a specific strategy, generate a complete and executable SAS code solution.
- The code should be syntactically correct and produce the desired output based on the query.
- Output ONLY the SAS code, without any explanations or additional text.
- Save all new dataframes in the WORK library (temporary).
- When doing union joins, select only relevant columns.
- Adhere to all coding instructions and guidelines specified below.

## DETAILS YOU HAVE TO KEEP IN MIND
in the following section we will outline the logic and structure of claims databases, as understanding how they operate is essential to write a correct query


**logic behind Structure of claims databases:**
<REDACTED>
Final NOTES :
<REDACTED>

**Coding rules**
<REDACTED>
"""

## UTILITY FUNCTIONS FOR THE AGENT

In [ ]:
def extract_final_answer(text):
    match = re.search(r"<final_answer>\n(.*?)</final_answer>", text, re.DOTALL)
    if match:
        content = match.group(1).strip()
        # Remove ```python if present
        content = content.replace("```python", "")
        # Remove ``` if present
        content = content.replace("```", "")
        return content.strip()

    # If no <final_answer> tags, check for ```python and </final_answer>
    match_python_block = re.search(r"```python\n(.*?)</final_answer>", text, re.DOTALL)
    if match_python_block:
        content = match_python_block.group(1).strip()
        # No need to remove ```python as it was the start marker
        # Remove ``` if present (might be closing a code block within)
        content = content.replace("```", "")
        return content.strip()

    # If neither pattern matches, return None
    return None

In [ ]:
import io
import sys
import contextlib

def execute_python_code(code_string):
  # Redirect stdout to capture output
  output_buffer = io.StringIO()
  with contextlib.redirect_stdout(output_buffer):
    try:
      # Execute the code
      exec(code_string)
    except Exception as e:
      # Capture error messages
      return f"Error: {e}"

  # Get the captured output
  output = output_buffer.getvalue()
  return output

## THE AGENT CODE

In [ ]:
def DATA_RETRIEVER(query, dev_mode = False):
    necessary_metadata = relevant_metadata_extraction(filtered_metadata, query, dev_mode)
    if necessary_metadata == None:
        return 'wrong choice of databases'

    from gradio_client import Client
    client = Client("Qwen/Qwen2.5-Max-Demo")

    prompt = f"""#**Metadata:**#{necessary_metadata}
    #**MEMORY:**#
    here is the memory of YOU, yes you, doing this task for questions that
    are eerily similar to the one you're dealing with now, learn from your past trials, successes and mistakes and make them
    your guide for wiser choices : {formulate_coding_guidance(query, memory_dict['coding_memory'])}
    #**query** (be faithful to it and dont forget the golden rule and silver rule, be faithful to the answer format)#{query}
    #answer part format : stay faithful to this format (
    only give python code (no seperate sas code) in this section without ```python in the beginning or ``` in the end or unnecessary indentation
    reason : we expect the put your python code in exec() command, be mindful of this when giving back ur final answer) :
<final_answer>
sas_code = "the sas code you've been contemplating here"
result = sas.submit(sas_code)
print(f"on a executé ce code : {{sas_code}}")
df = sas.sd2df(the name of the table containing the solution without 'work').to_excel('an adequate name to the extraction here.xlsx')
print(in french : tell the user where you uploaded the extraction + give a explanation of what you did and your approach)
</final_answer>"""
    chat_coder = client.chats.create(
    model=MODEL_ID,
    config=types.GenerateContentConfig(
        system_instruction=system_instruction_coder,
        temperature=0.5,
        ),
    )
    try:
        while True:
            response = chat_coder.send_message(prompt)
            extracted_code = extract_final_answer(response)
            if extracted_code != '' and extracted_code is not None:
                break
        if dev_mode:
            if extracted_code:
                # Ask the user for confirmation
                user_confirmation = input(f"Is the following code correct?\n{extracted_code}\n(yes/no): ").lower()

                if user_confirmation == "yes":
                    # Store the query-code pair in memory with status 'True'
                    # Check if the query already exists
                    if query in memory_dict['coding_memory']:
                        # Append the new response to the existing list of responses
                        memory_dict['coding_memory'][query].append({'status': True, 'response': extracted_code})
                    else:
                        # Create a new entry with a list containing the current response
                        memory_dict['coding_memory'][query] = [{'status': True, 'response': extracted_code}]

                    # Execute the code
                    exec(extracted_code)

                elif user_confirmation == "no":
                    # Ask the user for an explanation
                    user_explanation = input("Please explain why the code is not correct: ")

                    # Store information in memory with status 'False' and the user's explanation
                    # Check if the query already exists
                    if query in memory_dict['coding_memory']:
                        # Append the new response to the existing list of responses
                        memory_dict['coding_memory'][query].append({
                            'status': False,
                            'response': extracted_code,
                            'why no': user_explanation
                        })
                    else:
                        # Create a new entry with a list containing the current response
                        memory_dict['coding_memory'][query] = [{
                            'status': False,
                            'response': extracted_code,
                            'why no': user_explanation
                     }]

                    print("Stopping execution and saving feedback to memory.")
                    return None  # Stop execution

                else:
                    print("Invalid input. Please enter 'yes' or 'no'.")

            else:
                print("No code found in the response.")
            return None
        else:
            return execute_python_code(extracted_code)

    except Exception as e:
        print(f'\nError occurred:\nError message: {e}')

    return None

In [ ]:
print(DATA_RETRIEVER("""donne moi le type dossier afin et la charge des sinistres de survenance 2023""", dev_mode = False))

In [ ]:
import json
with open("D:/OneDrive - starAssurance/agent_memory.json", 'w') as f:
        json.dump(memory_dict, f, indent=4)